In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import markovify

In [2]:
import random
import string
wordlistStringString = []
counter = 0
trainingData = []
with open(r"D:\rockyou.txt\generatepasswords\csvRockyou.csv", 'r', encoding='utf-8') as file:    
    # Now read from the 100,000th line onward
    for line in file:
        # Process each line from here on
        wordlistStringString.append(line)
        wordlistStringString[counter] = wordlistStringString[counter].replace("\n", "")
        counter += 1
        if counter > 200000:
            trainingData.append(line)
            if counter == 400000:
                break
    

labels = []
for i in range(200000):
    labels.append(1)
for i in range(200000):
    labels.append(0)
test = []


In [3]:
import numpy as np
from collections import defaultdict
from scipy.sparse import dok_matrix, csr_matrix
import random
# Parameters
filename = r"C:\Users\1\Downloads\xato-net-10-million-passwords-1000000.txt"
num_rows = 100000  # Number of lines to read from rockyou.txt
min_length = 8
max_length = 12

# Step 1: Load Data
with open(filename, 'r', encoding="latin-1") as file:
    passwords = [next(file).strip() for _ in range(num_rows)]

# Step 2: Build Sparse Transition Matrix on a Word Level
# Map each unique word to an index
word_list = list(set(passwords))  # Unique words (passwords) in the dataset
word_to_index = {word: i for i, word in enumerate(word_list)}
index_to_word = {i: word for word, i in word_to_index.items()}
num_words = len(word_list)

# Initialize sparse matrix using Dictionary of Keys (DOK) for efficient building
transition_counts = dok_matrix((num_words, num_words), dtype=np.int32)

# Populate the sparse transition matrix with counts based on word sequences
for password in passwords:
    words = list(password)  # Treat each character of a password as a "word" for now
    for i in range(len(words) - 1):
        word1, word2 = words[i], words[i + 1]
        if word1 in word_to_index and word2 in word_to_index:
            transition_counts[word_to_index[word1], word_to_index[word2]] += 1

# Convert to CSR format for efficient row slicing
transition_counts = transition_counts.tocsr()

# Step 3: Normalize Counts to Probabilities
for i in range(num_words):
    row_sum = transition_counts[i, :].sum()
    if row_sum > 0:
        transition_counts[i, :] = transition_counts[i, :] / row_sum
print("loaded the file generating passwords...")
# Step 4: Generate Passwords Using Sparse Transition Matrix
import numpy as np
import random

passwords = []
# Sample list of passwords as training data
with open("C:\\Users\\1\\Downloads\\xato-net-10-million-passwords-1000000.txt", encoding="utf8") as f:
    for i in range(100000):
        passwords.append(f.readline().strip())

# Join passwords as a single string with a separator (optional)
text = " ".join(passwords)
print("loaded the passwords")

# Create a Markov model with bigram-level states (state_size=2)
model = markovify.Text(text, state_size=2)
bigrams = list(model.chain.model.keys())
# Function to generate a password by starting from a bigram
def generate_password(model, length=12):
    # Choose a random bigram from the model's keys to start
    bigram = random.choice(bigrams)
    password = ''.join(bigram)  # Initialize with the bigram

    # Generate the rest of the password
    while len(password) < length:
        state = tuple(password[-2:])  # Last two characters as the new state
        next_char_options = model.chain.model.get(state)

        if next_char_options:
            # Fast selection of the most frequent next character
            next_char = max(next_char_options, key=next_char_options.get)
            password += next_char[-1]  # Append only the last character
        else:
            # If we hit a dead-end, restart from a new random bigram
            bigram = random.choice(list(model.chain.model.keys()))
            password += ''.join(bigram)[:2]  # Append part of new bigram as fallback

    return password[:length]  # Ensure exact length


# Generate a batch of passwords

test = [generate_password(model, length=12) for _ in range(200000)]
# Generate a batch of passwords

for i in range(len(test)):
    trainingData.append(test[i])
for i in range(len(trainingData)):
    trainingData[i] = trainingData[i].replace("\n", "")

print(len(trainingData))

loaded the file generating passwords...
loaded the passwords
400000


In [4]:
zipped_data = list(zip(trainingData, labels))
print(zipped_data)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
# Importing necessary libraries
import numpy as np
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


# Assuming 'wordlistStringString' is your training data and 'labels' is the corresponding label array
vectorizer = HashingVectorizer(
    analyzer='char',
    n_features=2000000,  # Adjust the number of features (e.g., 2^20)
    ngram_range=(1, ),  # Use unigrams and bigrams
    dtype=np.float32,  # Use float32 for lower memory usage
    )
X_sparse = vectorizer.fit_transform(trainingData)
print("finished vectorizing features")
# Define the function to compute dense featuresimport numpy as np

import numpy as np
from scipy.sparse import csr_matrix

def compute_dense_features_vectorized(data):
    # Calculate the lengths of each word
    lengths = np.array([len(word) for word in data])
    
    # Calculate lowercase, uppercase, and digit counts for each word
    lowercases = np.array([sum(c.islower() for c in word) for word in data])
    uppercases = np.array([sum(c.isupper() for c in word) for word in data])
    digits = np.array([sum(c.isdigit() for c in word) for word in data])

    # Calculate the ratios
    digit_ratios = np.divide(digits, lengths, where=lengths != 0)
    letter_ratios = np.divide(lowercases + uppercases, lengths, where=lengths != 0)
    uppercase_ratios = np.divide(uppercases, lengths, where=lengths != 0)
    lowercase_ratios = np.divide(lowercases, lengths, where=lengths != 0)

    # Stack all the features into a single matrix
    return np.column_stack((lengths, lowercases, uppercases, digits, digit_ratios, letter_ratios, uppercase_ratios, lowercase_ratios))
print("computing dense features")
# Compute dense features for the entire list at once
dense_features = compute_dense_features_vectorized(trainingData)

# Convert dense features to a sparse matrix format
dense_features_sparse = csr_matrix(dense_features)
print(len(labels))
print(len(trainingData))
print(zip(trainingData, labels))
# Combine the sparse hashed text features with the dense numeric features
X_combined = hstack([X_sparse, dense_features_sparse])
print("finished computing features, training the model...")
# Continue with model training and evaluation as before
classifier = LogisticRegression(  # Optimized for large datasets and sparse data
    penalty='l1',           # L2 regularization
    solver='saga',          # Optimized for large, sparse datasets
    max_iter=100000,          # Higher iterations for convergence
    C=1.0,                  # Regularization strength (adjust as needed)
    random_state=42  
)
classifier.fit(X_combined, labels)

# Evaluate the model
predictions = classifier.predict(X_combined)
report = classification_report(labels, predictions)
print(report)


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
generatedPasswords = []
numOfPasswords = 1000000
print(f"generatiing {numOfPasswords} passwords")
for i in range(numOfPasswords):
    generatedPasswords.append(generate_password(model, length=12))
print(f"generated: {numOfPasswords} passwords")
    
# Vectorize training data and compute dense features
print("vectorizing Data")
vectorizer = HashingVectorizer(
    analyzer='char',
    n_features=2000000,  # Adjust the number of features (e.g., 2^20)
    ngram_range=(1, 6),  # Use unigrams and bigrams
    dtype=np.float32,  # Use float32 for lower memory usage
    )
X_train_sparse = vectorizer.fit_transform(generatedPasswords)
print("finished vectorizing")
print("computing features")
dense_train_features = compute_dense_features_vectorized(generatedPasswords)
dense_train_sparse = csr_matrix(dense_train_features)

print(dense_train_sparse.shape)
print(X_train_sparse.shape)
# Combine sparse n-grams with dense features for test data
X_test_combined = hstack([X_train_sparse, dense_train_sparse])


# Predict using the classifier
print("finished computing features. predicting passwords...")
predictions = classifier.predict(X_test_combined)

# Output the results
count_ones = np.sum(predictions == 1)
count_zeros = np.sum(predictions == 0)
print("Number of 1s:", count_ones)
print("Number of 0s:", count_zeros)

# Print predicted potential passwords
for i, prediction in enumerate(predictions):
    if prediction == 1:
        print(test[i])


In [ ]:
classifier.coef_

In [ ]:
trainingData = final_df.drop(columns=["password", "bigram_ids", "trigram_ids"])

In [ ]:
labels = kmeans.predict(trainingData)


In [ ]:
unique_values, counts = np.unique(labels, return_counts=True)

In [ ]:
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

In [ ]:
randomS = []

In [ ]:
for i in range(20000):
    length = random.randint(8, 12)
       # Ensure the first character is a letter, either uppercase or lowercase
    first_char = random.choice(string.ascii_letters[:26] + string.ascii_letters[26:])  # First letter can be upper or lower
    
    # Generate the rest of the string with lowercase letters and digits
    remaining_chars = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(length - 1))
    
    wordlistStringString.append(first_char + remaining_chars)

In [ ]:
processedData = process_passwords(randomS)

In [ ]:
processedData.columns

In [ ]:
tryingLabels = kmeans.predict(processedData)

In [ ]:
unique_values, counts = np.unique(tryingLabels, return_counts=True)

In [ ]:
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

In [ ]:
processedData